In [4]:
#General Library
import numpy as np
import pandas as pd
import time
import datetime
import random
import os
import matplotlib.pyplot as plt

#MachinLearning Library
import lightgbm as lgb
import sklearn
from sklearn.model_selection import train_test_split

#Set Random Seed
seed=777
os.environ['PYTHONHASHED']=str(seed)
np.random.seed(seed)
random.seed(seed)

In [5]:
# Print Information
print('Seed: %i'%(seed))
print('Numpy: %s'%(np.__version__))
print('Pandas: %s'%(pd.__version__))
print('LightGBM: %s'%(lgb.__version__))
print('Scikit-Learn: %s'%(sklearn.__version__))

Seed: 777
Numpy: 1.18.5
Pandas: 1.1.3
LightGBM: 3.1.1
Scikit-Learn: 0.23.2


In [35]:
def sort_dataset(dataset):
    '''
    This function sorts the meteric_id of train.csv and test.csv into numerical order.
    '''
    columns=dataset.columns
    meter_ids=columns[1:] 
    tmp=[]
    for meter_id in meter_ids:
        meter_id=meter_id.replace('NX','')   #'NX'???
        tmp.append(int(meter_id)) ##int???
    tmp=np.sort(tmp)
    
    meter_ids=[]
    for meter_id in tmp:
        meter_id = 'X'+str(meter_id)
        meter_ids.append(meter_id)
        
    results=[dataset[columns[0]].values]
    for meter_id in meter_ids:
        values=dataset[meter_id].values
        results.append(values)
    results=np.array(results).T
    df=pd.DataFrame(results, columns=[columns[0]]+meter_ids)
    return df

In [36]:
d=pd.read_csv('train.csv')
print(d)

                   Time    NX1    NX2    NX3    NX4  NX5    NX6    NX7    NX8  \
0      2016-07-26 11:00    NaN    NaN    NaN    NaN  NaN    NaN    NaN    NaN   
1      2016-07-26 12:00    NaN    NaN    NaN    NaN  NaN    NaN    NaN    NaN   
2      2016-07-26 13:00    NaN    NaN    NaN    NaN  NaN    NaN    NaN    NaN   
3      2016-07-26 14:00    NaN    NaN    NaN    NaN  NaN    NaN    NaN    NaN   
4      2016-07-26 15:00    NaN    NaN    NaN    NaN  NaN    NaN    NaN    NaN   
...                 ...    ...    ...    ...    ...  ...    ...    ...    ...   
16904  2018-06-30 19:00  0.271  0.754  0.327  0.468  NaN  0.314  0.769  0.790   
16905  2018-06-30 20:00  0.341  0.827  0.478  0.356  NaN  0.434  0.680  0.819   
16906  2018-06-30 21:00  0.535  0.516  0.558  0.496  NaN  0.479  0.590  0.910   
16907  2018-06-30 22:00  0.379  0.494  0.605  0.320  NaN  0.555  0.604  1.044   
16908  2018-06-30 23:00  0.388  0.482  0.651  0.284  NaN  0.452  0.629  1.116   

         NX9  ...  NX1291  

In [37]:
#Load data
train=sort_dataset(pd.read_csv('train.csv'))
test=sort_dataset(pd.read_csv('test.csv'))

KeyError: 'X1'

In [42]:
pandas.fillna?

Object `pandas.fillna` not found.


### 데이터 전처리
1. train,test의 결측치 모두 0으로 변경
2. 날짜 타입을 datetime으로 변경
3. 시간별 날씨 -> 평균을 이용해 -> 일자별 날씨로 변경 

In [26]:
def time_convert(df_time,string_type='train'):
    '''
    This function changes format of time from string to datetime.
    '''
    old_times=df_time
    new_times=old_times.copy()
    for i, old_time in enumerate(old_times):
        if string_type == 'train':
            new_time = datetime.datetime.strptime(old_time, '%Y-%m-%d %H:%M' )
        elif string_type == 'test':
            new_time=datetime.datetime.strptime(old_time, '%Y.%m.%d %H:%M')
        else:
            new_time = datetime.datetime.strptime(old_time, '%Y-%m-%d')
        new_times[i] = new_time
    return new_times

def split_day(_times, _dates):
    '''
    This function splits power consumption data and weather data by days.
    '''
    for time in _times:
        if time.time().hour==0: #time.time().hour???
            ref_time=time.date()  #time.date()????
            break
    times=[]
    datas=[]
    data_tmp=[]
    for i,time in enumerate(_times):
        time=time.date()
        data=_datas[i]
        if ref_time > time: #&gt;??? = 부등호 >
            pass
        elif ref_time == time:
            data_tmp.append(data)
        else:
            times.append(ref_time)
            datas.append(data_tmp)
            ref_time = time
            data_tmp = [data]
    if ref_time not in times:
        if len(data_tmp) == 24:
            times.append(ref_time)
            datas.append(data_tmp)
    times = np.array(times)
    datas = np.array(datas)
    return times, datas
            

In [16]:
time?

Type:        module
String form: <module 'time' (built-in)>
Docstring:  
This module provides various functions to manipulate time values.

There are two standard representations of time.  One is the number
of seconds since the Epoch, in UTC (a.k.a. GMT).  It may be an integer
or a floating point number (to represent fractions of seconds).
The Epoch is system-defined; on Unix, it is generally January 1st, 1970.
The actual value can be retrieved by calling gmtime(0).

The other representation is a tuple of 9 integers giving local time.
The tuple items are:
  year (including century, e.g. 1998)
  month (1-12)
  day (1-31)
  hours (0-23)
  minutes (0-59)
  seconds (0-59)
  weekday (0-6, Monday is 0)
  Julian day (day in the year, 1-366)
  DST (Daylight Savings Time) flag (-1, 0 or 1)
If the DST flag is 0, the time is given in the regular time zone;
if it is 1, the time is given in the DST time zone;
if it is -1, mktime() should guess based on the date and time.


In [28]:
#Replace nan to zero
train=train.replace(np.nan, 0.0)
test=test.replace(np.nan,0.0)

#Convert time data format to datetime
train_times=time_convert(train['Time'], string_type='train').values
test_times = time_convert(test['Time'], string_type='test').values
#Meter id
train_meter_ids = train.columns[1:]
test_meter_ids=test.columns[1:]

NameError: name 'train' is not defined

In [ ]:
# Downsampling (a day)
temperature = np.mean(split_day(add_times, temperature)[1], axis=1)
rainfall = np.mean(split_day(add_times, rainfall)[1], axis=1)
wind = np.mean(split_day(add_times, wind)[1], axis=1)
humidity = np.mean(split_day(add_times, humidity)[1], axis=1)
snowfall = np.mean(split_day(add_times, snowfall)[1], axis=1)
add_times, cloud = split_day(add_times, cloud)
cloud = np.mean(cloud, axis=1)

# Make additional data set
additional_data = np.array([   
    add_times,
    temperature,
    rainfall,
    wind,
    humidity,
    snowfall,
    cloud,    
])

#### 3. 탐색적 자료분석
Exploratory Data Analysis
 test.csv는 id 481번 이하로 분포
 모델 생성 시 0 ~ 481번 데이터만 사용하는 것을 고려

In [ ]:
train_id_num = []
train_data_num = []
for meter_id in train_meter_ids:
    meter_num = int(meter_id.replace('X',''))
    valid_num = len(np.where(train[meter_id]) > 0.0)[0])
    train_id_num.append(meter_num)
    train_data_num.append(valid_num)
test_id_num=[]
test_data_num=[]
for meter_id in test_meter_ids:
    meter_num = int(meter_id.replace('X',''))
    valid_num = len(np.where(test[meter_id] > 0.0)[0]) # &gt; ???
    test_id_num.append(meter_num)
    test_data_num.append(valid_num)

plt.scatter(train_id_num, train_data_num, s=5, color='b', label='train')
plt.scatter(test_id_num, test_data_num, s=5, color='r', label='test')
plt.legend()
plt.xlim(0, 1500)
plt.ylim(bottom=0)
plt.xlabel('Meter Id')
plt.ylabel('Data Numer')
plt.show()

In [ ]:

def hansu(a):
    count=0
    for a in range(1,a+1):
        if a <= 99:
           count+=1

        else:
            lista=list(str(a)) #[1,2,3,4,6]
            for i in range(1,len(lista)): #12346
                if int(lista[i])-int(lista[i-1])==int(lista[1])-int(lista[0]):
                    han=1
                else:
                    han=0
                    break
            if han==1:
                count+=1
    print(count)

hansu(100)

In [ ]:
## 4. 변수 선택 및 모델 구축
